In [18]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///croot/scikit-learn_1684954695550/work


In [19]:
!python -V

Python 3.11.7


In [20]:
import pickle
import os
import numpy as np
import pandas as pd

In [21]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/ivan/anaconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ivan/anaconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [22]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [23]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [24]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [25]:
std_y_pred = np.std(y_pred)
print("y_pred std:", std_y_pred)

y_pred std: 6.247488852238703


In [26]:
year = 2023
month = 3  

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [27]:
df_result = pd.DataFrame()
df_result['ride_id']=df['ride_id']
df_result['prediction_duration']= y_pred

output_fp = "output/yellow_tripdata_2023-03.parquet"

df_result.to_parquet(
    output_fp,
    engine='pyarrow',
    compression=None,
    index=False
)

In [28]:
print("file size:", os.stat(output_fp).st_size)

file size: 68641903
